## Colab Session 연장

### 아래 코드를 F12를 누른 후 Console에 입력할것 

function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);

# CLI 환경 실행

In [5]:
!pip install kora
from kora import console
console.start()  # and click link

     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
Console URL: https://teleconsole.com/s/ec0ca31af5ac9bf3c870461852f35c3e1cfabbdd


In [6]:
!nvidia-smi

Sun Mar  7 08:07:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!git clone https://github.com/jsk1107/dacon_k_fashion

Cloning into 'dacon_k_fashion'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 5), reused 15 (delta 3), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [1]:
!wget dacon-datasets.s3.ap-northeast-2.amazonaws.com/k-fashion/data.zip

--2021-03-07 07:41:33--  http://dacon-datasets.s3.ap-northeast-2.amazonaws.com/k-fashion/data.zip
Resolving dacon-datasets.s3.ap-northeast-2.amazonaws.com (dacon-datasets.s3.ap-northeast-2.amazonaws.com)... 52.219.56.15
Connecting to dacon-datasets.s3.ap-northeast-2.amazonaws.com (dacon-datasets.s3.ap-northeast-2.amazonaws.com)|52.219.56.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5306730245 (4.9G) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   4.94G  23.5MB/s    in 3m 35s  

2021-03-07 07:45:09 (23.5 MB/s) - ‘data.zip’ saved [5306730245/5306730245]



In [4]:
!unzip data.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/k/k34p5w0u3l.jpg  
  inflating: train/k/kvbi74cokl.jpg  
  inflating: train/k/k0kdnqn7oc.jpg  
  inflating: train/k/kq2klj34em.jpg  
  inflating: train/k/k6qi684hz8.jpg  
  inflating: train/k/k44jkrxipu.jpg  
  inflating: train/k/k82gtehhoh.jpg  
  inflating: train/k/ka7ehpv0hb.jpg  
  inflating: train/k/kbapodc6j3.jpg  
  inflating: train/k/k823dcnx9d.jpg  
  inflating: train/k/kjon7vscdy.jpg  
  inflating: train/k/krh658w9t9.jpg  
  inflating: train/k/k4wdi5fnth.jpg  
  inflating: train/k/ki44zgyay9.jpg  
  inflating: train/k/kepwo6ha91.jpg  
  inflating: train/k/k6w0cl3j2y.jpg  
  inflating: train/k/k36pp61464.jpg  
  inflating: train/k/kxesnveyl8.jpg  
  inflating: train/k/kcj1z5ngx4.jpg  
  inflating: train/k/k5555wxt81.jpg  
  inflating: train/k/kztd1gym0z.jpg  
  inflating: train/k/karxyihcu5.jpg  
  inflating: train/k/kc4hj5q4qh.jpg  
  inflating: train/k/kuugf2dlb4.jpg  
  inflating: train/k/ka16f0s1gz.jpg  
  inflating: t

In [16]:
!pip install -r /content/dacon_k_fashion/requrirement.txt

     |████████████████████████████████| 735.5MB 23kB/s 
     |████████████████████████████████| 17.3MB 252kB/s 
  Found existing installation: torch 1.7.1+cu101
    Uninstalling torch-1.7.1+cu101:
      Successfully uninstalled torch-1.7.1+cu101
  Found existing installation: torchvision 0.8.2+cu101
    Uninstalling torchvision-0.8.2+cu101:
      Successfully uninstalled torchvision-0.8.2+cu101


# Import

In [2]:
import torch
import torch.nn as nn
from torchvision.models.detection import retinanet_resnet50_fpn
from torch.utils.data import DataLoader
from torchvision.transforms import transforms as T
from torchvision.models.detection.retinanet import RetinaNetClassificationHead
from torch.utils.data import Dataset
from pycocotools.coco import COCO
import os
import cv2
from PIL import Image
import numpy as np
from pycocotools.cocoeval import COCOeval

In [3]:
!pip list

Package                       Version       
----------------------------- --------------
absl-py                       0.10.0        
alabaster                     0.7.12        
albumentations                0.1.12        
altair                        4.1.0         
appdirs                       1.4.4         
argon2-cffi                   20.1.0        
asgiref                       3.3.1         
astor                         0.8.1         
astropy                       4.2           
astunparse                    1.6.3         
async-generator               1.10          
atari-py                      0.2.6         
atomicwrites                  1.4.0         
attrs                         20.3.0        
audioread                     2.1.9         
autograd                      1.3           
Babel                         2.9.0         
backcall                      0.2.0         
beautifulsoup4                4.6.3         
bleach                        3.3.0         
blis      

# DataLoader

In [4]:
class KfashionDataset(Dataset):
    def __init__(self, img_dir, anno_dir, set_name='train', transforms=None):
        self.img_dir = img_dir
        self.anno_dir = anno_dir
        self.transforms = transforms

        self.coco = COCO(os.path.join(self.anno_dir, f'{set_name}.json'))

        self.image_ids = self.coco.getImgIds()
        self.category = self.coco.loadCats(self.coco.getCatIds())

    def __getitem__(self, ids):

        img = self.get_image(ids)
        target = self.get_annotaion(ids)

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.image_ids)

    def get_image(self, ids):
        img_info = self.coco.loadImgs(self.image_ids[ids])[0]
        key = list(img_info['file_name'])[0]
        img_path = os.path.join(self.img_dir, key, img_info['file_name'])
        img = Image.open(img_path)
        # img = cv2.imread(img_path)

        return img

    def get_annotaion(self, ids):
        anno_ids = self.coco.getAnnIds(imgIds=self.image_ids[ids], iscrowd=False)
        coco_annos = self.coco.loadAnns(anno_ids)
        target = {}
        boxes = []
        labels = []
        for coco_anno in coco_annos:
            bbox = self.bbox_transform(coco_anno['bbox'])
            boxes.append(bbox)
            labels.append(coco_anno['category_id']-1)
        target['boxes'] = torch.as_tensor(boxes, dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)

        return target

    def bbox_transform(self, bbox):
        bbox[2] = bbox[0] + bbox[2]
        bbox[3] = bbox[1] + bbox[3]
        return bbox

In [4]:
print('cuda check')
print(torch.cuda.is_available())
  

cuda check
True


# Train

In [7]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # (역자주: 학습시 50% 확률로 학습 영상을 좌우 반전 변환합니다)
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


img_dir = '/content/train/'
anno_dir = '/content/'
set_name = 'train'
dataset = KfashionDataset(img_dir, anno_dir, set_name, transforms=get_transform(True))
num_classes = len(dataset.category)
print(f'num_classes: {num_classes}')

dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)

model = retinanet_resnet50_fpn(True)
num_anchor = model.anchor_generator.num_anchors_per_location()[0]
in_channels = model.head.classification_head.cls_logits.in_channels
model.head.classification_head = RetinaNetClassificationHead(in_channels, num_anchor, num_classes)

optim = torch.optim.SGD(model.parameters(), lr=1e-4)

model.train()
for img, targets in dataloader:
    optim.zero_grad()

    if torch.cuda.is_available():
      print('cuda is available')
      model = model.cuda()
      img = img.cuda()
    out = model(img, targets)

    cls_loss = out['classification']
    reg_loss = out['bbox_regression']

    loss = cls_loss + reg_loss
    print(f'loss: {loss}')

    loss.backward()
    optim.step()

loading annotations into memory...
Done (t=2.77s)
creating index...
index created!
num_classes: 21
loss: 1.59769868850708


KeyboardInterrupt: ignored